**Addressing Overfitting in the 1st Model**

We will implement the following steps to mitigate overfitting:

1. **Expand the Dataset:** Gather additional data to provide the model with more diverse examples.
2. **Simplify the Model:** Reduce complexity by removing unnecessary nodes or layers from the neural network architecture.
3. **Apply Regularization:** Employ techniques like L1 or L2 regularization to penalize large weights and prevent overfitting.
4. **Feature Selection:** Carefully choose relevant features and eliminate those that might contribute to overfitting.

In this model, we will use a simplified model and apply regularization to improve accuracy on the test set.  
If it doesn't work well, I will try to collect more data from other datasets.  
I am not using feature selection methods because feature extraction is already handled in CNNs by Conv2D and MaxPooling2D layers.

# Model 2

## 1. Import library

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

## 2. Loading dataset

In [2]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')

data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)

for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)


In [3]:
data_set = np.array(data_set)
label = np.array(label)
data_set.shape

(1092, 64, 64, 3)

## 3. Split and normalize data


In [5]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)


In [6]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )
x_train,x_val,y_train,y_val = train_test_split(
        x_train,y_train,
    test_size=0.25,
    random_state=99
)
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape}\nX validation shape: {x_val.shape}\nY validation shape: {x_val.shape}')

X train shape: (654, 64, 64, 3)
Y train shape: (654,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,)
X validation shape: (219, 64, 64, 3)
Y validation shape: (219, 64, 64, 3)


In [7]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)
x_val = normalize(x_val, axis=1)

## 4. Building model

In [8]:
from keras.models import Sequential 
from tensorflow.keras.regularizers import l2
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    Dropout,
    Flatten,
    Dense
)

In [9]:
INPUT_SIZE = 64
weight_decay = 1e-4

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3), kernel_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Layer
model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform', kernel_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(64, kernel_regularizer=l2(weight_decay)))
model.add(Activation('relu'))
model.add(Dropout(0.8))

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

e:\Anaconda\Anaconda setup\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 5. Training model

In [11]:
from tensorflow.keras.optimizers import Adam
# Compile the model
model.compile(
    optimizer=Adam(amsgrad=True),
    loss='binary_crossentropy', 
    metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 29, 29, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 176,225 (688.38 KB)

 Trainable params: 176,225 (688.38 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    verbose=1,
    epochs=10,
    validation_data=(x_val,y_val),
    shuffle=False
    )

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.4964 - loss: 0.7326 - val_accuracy: 0.6758 - val_loss: 0.6919
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6850 - loss: 0.6593 - val_accuracy: 0.7671 - val_loss: 0.5185
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7650 - loss: 0.5135 - val_accuracy: 0.8174 - val_loss: 0.4372
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8265 - loss: 0.4169 - val_accuracy: 0.8493 - val_loss: 0.3717
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8897 - loss: 0.3301 - val_accuracy: 0.9041 - val_loss: 0.2974
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9157 - loss: 0.2716 - val_accuracy: 0.9178 - val_loss: 0.2698
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9403 - loss: 0.2321 - val_accuracy: 0.8995 - val_loss: 0.2807
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9407 - loss: 0.1972 - val_accuracy: 0.9269 - v

## 6. Test with new data

In [16]:
y_test_pred = model.predict(x_test)
y_pred = (y_test_pred >0.5).astype(int)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [17]:
y_test_reshape = y_test.reshape(-1,1)
print("Accuracy in test set:", accuracy_score(y_test_reshape, y_pred))
print('Accuracy in validation set:',history.history['val_accuracy'][-1])

Accuracy in test set: 0.9452054794520548
Accuracy in validation set: 0.9360730648040771


**This model gain 94.5% accuracy in test set. Now it's good to use!**

## Save model

**Infor**<br>
Name: BrainTurmor_v2<br>
Accuracy in test set: 0.9452054794520548<br>
Accuracy in validation set: 0.9360730648040771<br>
Status: Good

In [18]:
model.save('BrainTurmor_v2.keras')